In [1]:
import numpy as np 
import pandas as pd 
import networkx as nx


#df0 = pd.read_csv('train.csv',sep=',')
#df1 = pd.read_csv('test.csv',sep=',')

In [2]:
#df = df0.sample(n=6000)
#df.to_csv('DataFrame')

In [3]:
df = pd.read_csv('DataFrame', sep=',')

In [4]:
df

,Unnamed: 0,Unnamed: 0.1,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,28277,28277,54732,Female,Loyal Customer,69,Personal Travel,Eco,964,3,...,5,5,3,5,1,5,3,0,0.0,neutral or dissatisfied
1,78830,78830,90725,Male,Loyal Customer,55,Business travel,Business,240,2,...,1,1,2,3,2,3,2,163,153.0,neutral or dissatisfied
2,27414,27414,24654,Female,Loyal Customer,51,Business travel,Business,2672,5,...,4,4,4,4,5,4,3,0,0.0,satisfied
3,72198,72198,31150,Male,disloyal Customer,27,Business travel,Eco,495,2,...,3,4,4,3,4,2,3,0,0.0,neutral or dissatisfied
4,191,191,2980,Male,Loyal Customer,12,Personal Travel,Eco,462,2,...,2,3,4,2,2,1,2,0,0.0,neutral or dissatisfied
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,12805,12805,92838,Female,Loyal Customer,49,Personal Travel,Eco,1587,3,...,4,4,3,5,3,4,5,35,12.0,neutral or dissatisfied
5996,13956,13956,47468,Female,Loyal Customer,33,Personal Travel,Eco,588,2,...,5,5,3,5,5,5,5,0,0.0,neutral or dissatisfied
5997,1632,1632,53491,Male,Loyal Customer,50,Personal Travel,Eco,2358,1,...,4,4,5,4,4,3,4,0,0.0,neutral or dissatisfied
5998,78050,78050,29098,Male,Loyal Customer,35,Personal Travel,Eco,672,2,...,5,1,1,4,4,1,5,0,0.0,neutral or dissatisfied


In [5]:
# Check for missing data
df.isna().sum()

Unnamed: 0                            0
Unnamed: 0.1                          0
id                                    0
Gender                                0
Customer Type                         0
Age                                   0
Type of Travel                        0
Class                                 0
Flight Distance                       0
Inflight wifi service                 0
Departure/Arrival time convenient     0
Ease of Online booking                0
Gate location                         0
Food and drink                        0
Online boarding                       0
Seat comfort                          0
Inflight entertainment                0
On-board service                      0
Leg room service                      0
Baggage handling                      0
Checkin service                       0
Inflight service                      0
Cleanliness                           0
Departure Delay in Minutes            0
Arrival Delay in Minutes             16


In [6]:
df.head()

,Unnamed: 0,Unnamed: 0.1,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,28277,28277,54732,Female,Loyal Customer,69,Personal Travel,Eco,964,3,...,5,5,3,5,1,5,3,0,0.0,neutral or dissatisfied
1,78830,78830,90725,Male,Loyal Customer,55,Business travel,Business,240,2,...,1,1,2,3,2,3,2,163,153.0,neutral or dissatisfied
2,27414,27414,24654,Female,Loyal Customer,51,Business travel,Business,2672,5,...,4,4,4,4,5,4,3,0,0.0,satisfied
3,72198,72198,31150,Male,disloyal Customer,27,Business travel,Eco,495,2,...,3,4,4,3,4,2,3,0,0.0,neutral or dissatisfied
4,191,191,2980,Male,Loyal Customer,12,Personal Travel,Eco,462,2,...,2,3,4,2,2,1,2,0,0.0,neutral or dissatisfied


In [7]:
df.drop(columns=['Unnamed: 0', 'Unnamed: 0.1','id'],inplace=True)

In [8]:
df.head()

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,Female,Loyal Customer,69,Personal Travel,Eco,964,3,2,3,3,...,5,5,3,5,1,5,3,0,0.0,neutral or dissatisfied
1,Male,Loyal Customer,55,Business travel,Business,240,2,5,5,5,...,1,1,2,3,2,3,2,163,153.0,neutral or dissatisfied
2,Female,Loyal Customer,51,Business travel,Business,2672,5,5,5,5,...,4,4,4,4,5,4,3,0,0.0,satisfied
3,Male,disloyal Customer,27,Business travel,Eco,495,2,3,2,2,...,3,4,4,3,4,2,3,0,0.0,neutral or dissatisfied
4,Male,Loyal Customer,12,Personal Travel,Eco,462,2,4,2,3,...,2,3,4,2,2,1,2,0,0.0,neutral or dissatisfied


In [9]:
X = df.copy(deep=True)

In [10]:
X.satisfaction.unique()

array(['neutral or dissatisfied', 'satisfied'], dtype=object)

In [11]:
X.Gender = X.Gender.apply(lambda x: 0 if x == 'Male' else 1)
X['Customer Type'] = X['Customer Type'].apply(lambda x: 0 if x == 'Loyal Customer' else 1)
X['Type of Travel'] = X['Type of Travel'].apply(lambda x: 0 if x == 'Personal Travel' else 1)
X['Class'] = X['Class'].apply(lambda x: 0 if x == 'Eco' else(1 if x == 'Eco Plus' else 2))
X['satisfaction'] = X['satisfaction'].apply(lambda x: 0 if x == 'neutral or dissatisfied' else 1)

In [12]:
X.reset_index(drop=True,inplace=True)
X

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,1,0,69,0,0,964,3,2,3,3,...,5,5,3,5,1,5,3,0,0.0,0
1,0,0,55,1,2,240,2,5,5,5,...,1,1,2,3,2,3,2,163,153.0,0
2,1,0,51,1,2,2672,5,5,5,5,...,4,4,4,4,5,4,3,0,0.0,1
3,0,1,27,1,0,495,2,3,2,2,...,3,4,4,3,4,2,3,0,0.0,0
4,0,0,12,0,0,462,2,4,2,3,...,2,3,4,2,2,1,2,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,1,0,49,0,0,1587,3,4,3,4,...,4,4,3,5,3,4,5,35,12.0,0
5996,1,0,33,0,0,588,2,5,2,3,...,5,5,3,5,5,5,5,0,0.0,0
5997,0,0,50,0,0,2358,1,4,1,5,...,4,4,5,4,4,3,4,0,0.0,0
5998,0,0,35,0,0,672,2,3,2,2,...,5,1,1,4,4,1,5,0,0.0,0


In [13]:
X.drop(columns=['Arrival Delay in Minutes'],inplace = True)

In [14]:
# max number of graph generation features
np.log(len(X))

8.699514748210191

In [15]:
X.head()

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,...,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,satisfaction
0,1,0,69,0,0,964,3,2,3,3,...,2,5,5,3,5,1,5,3,0,0
1,0,0,55,1,2,240,2,5,5,5,...,2,1,1,2,3,2,3,2,163,0
2,1,0,51,1,2,2672,5,5,5,5,...,1,4,4,4,4,5,4,3,0,1
3,0,1,27,1,0,495,2,3,2,2,...,3,3,4,4,3,4,2,3,0,0
4,0,0,12,0,0,462,2,4,2,3,...,2,2,3,4,2,2,1,2,0,0


In [16]:
X.columns

Index(['Gender', 'Customer Type', 'Age', 'Type of Travel', 'Class',
       'Flight Distance', 'Inflight wifi service',
       'Departure/Arrival time convenient', 'Ease of Online booking',
       'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
       'Inflight entertainment', 'On-board service', 'Leg room service',
       'Baggage handling', 'Checkin service', 'Inflight service',
       'Cleanliness', 'Departure Delay in Minutes', 'satisfaction'],
      dtype='object')

Following variables are used for the graph generation:

- Gender
- Customer Type
- Age
- Type of Travel
- Class

In [17]:
X.Age.describe()

count    6000.000000
mean       39.101000
std        15.172931
min         7.000000
25%        27.000000
50%        40.000000
75%        50.000000
max        85.000000
Name: Age, dtype: float64

In [17]:
X.to_csv('clean_data')

In [18]:
genvars = X[['Customer Type','Age','Type of Travel','Class']].copy(deep=True)

In [19]:
genvars.Age = genvars.Age.apply(lambda x: 0 if x < 26 else(1 if x < 39 else(2 if x < 50 else 3)))

In [20]:
genvars

,Customer Type,Age,Type of Travel,Class
0,0,3,0,0
1,0,3,1,2
2,0,3,1,2
3,1,1,1,0
4,0,0,0,0
...,...,...,...,...
5995,0,2,0,0
5996,0,1,0,0
5997,0,3,0,0
5998,0,1,0,0


In [21]:
# gender
gend_alpha0 = 0.60
gend_beta1 = 0.40

aff_gend = np.array([[gend_alpha0,gend_beta1],
                     [gend_beta1,gend_alpha0]])

# customer type (loyal vs disoyal)

type_alpha0 = 0.8
type_beta1 = 0.5

aff_type = np.array([[type_alpha0, type_beta1],
                     [type_beta1, type_alpha0]])


# age
age_alpha0 = 0.90
age_beta1 = 0.80
age_beta2 = 0.60
age_beta3 = 0.40

aff_age = np.array([[age_alpha0,age_beta1,age_beta2,age_beta3],
                    [age_beta1,age_alpha0,age_beta1,age_beta2],
                    [age_beta2,age_beta1,age_alpha0,age_beta1],
                    [age_beta3,age_beta2,age_beta1,age_alpha0]])


# Type of travel
trav_type_alpha0 = 0.80
trav_type_beta1 = 0.20

aff_trav_type = np.array([[trav_type_alpha0, trav_type_beta1],
                          [trav_type_beta1, trav_type_alpha0]])

# Class

class_alpha0 = 0.85
class_beta1 = 0.60
class_beta2 = 0.40

aff_class = np.array([[class_alpha0, class_beta1, class_beta2],
                      [class_beta1, class_alpha0, class_beta1],
                      [class_beta2, class_beta1, class_alpha0]])


aff = {0:,0:aff_type, 1:aff_age, 2:aff_trav_type, 3:aff_class}


In [22]:
x = genvars.to_numpy()
x = x.astype(int)
x = x.T

P_u_v = np.empty((len(x.T),len(x.T)))
prob_u_v = np.empty((len(x),1))
u = np.empty((len(x),1))
v = np.empty((len(x),1))

# create probability for connection between u and v
for i in range(len(x.T)):
    u = x[:,i]

    for k in range(len(x.T)):
        v = x[:,k]

        for j in range(len(x)):
            aff_current = aff[j]
            prob_u_v[j] = aff_current[u[j],v[j]]

        P_u_v[i,k] = np.prod(prob_u_v)

In [23]:
A = np.triu(P_u_v,1)

for i in range(len(P_u_v)):
    for j in range(len(P_u_v)):
        if A[i,j] > np.random.rand():
            A[i,j] = 1
        else:
            A[i,j] = 0


In [24]:
A = A + A.T

In [25]:
np.savetxt('adjacency_matrix_no_gender.csv', A, delimiter=',')

In [28]:
A = np.loadtxt('adjacency_matrix.csv',delimiter=',')

In [29]:
G = nx.from_numpy_matrix(A)

In [32]:
from node2vec import Node2Vec

node2vec = Node2Vec(G, dimensions=2, walk_length=8, num_walks=100, workers=1)
model = node2vec.fit(window=10, min_count=1, batch_words=2)
#model.wv.save_word2vec_format('embeddings')

Generating walks (CPU: 1): 100%|██████████| 100/100 [18:30:18<00:00, 666.18s/it]     


In [29]:
nx.classes.function.density(G)

0.09377457354003445

In [ ]:
cent = nx.algorithms.centrality.eigenvector_centrality(G)
clos_cent = nx.algorithms.centrality.closeness_centrality(G)
be_cent = nx.algorithms.centrality.betweenness_centrality(G)

In [ ]:
obj = cent.items()
l = list(obj)
r = np.array(l)
r = r[:,1]

In [ ]:
plt.hist(r)
plt.title('Eigenvector Centrality')
plt.xlabel('Eigenvector Centrality')
plt.ylabel('Number of Nodes')

In [ ]:
obj = clos_cent.items()
l = list(obj)
r = np.array(l)
r = r[:,1]

In [ ]:
plt.hist(r)
plt.title('Closeness Centrality')
plt.xlabel('Closeness Centrality')
plt.ylabel('Number of Nodes')

In [ ]:
obj = be_cent.items()
l = list(obj)
r = np.array(l)
r = r[:,1]

In [ ]:
plt.hist(r)
plt.title('Betweenness Centrality')
plt.xlabel('Betweenness Centrality')
plt.ylabel('Number of Nodes')

In [ ]:
import collections


degree_sequence = sorted([d for n, d in G.degree()], reverse=True)  # degree sequence
degreeCount = collections.Counter(degree_sequence)
deg, cnt = zip(*degreeCount.items())

fig, ax = plt.subplots(figsize=(20,20))
plt.bar(deg, cnt, width=0.80, color="b")

plt.title("Degree Histogram")
plt.ylabel("Count")
plt.xlabel("Degree")
ax.set_xticks([d + 0.4 for d in deg])
ax.set_xticklabels(deg)

# draw graph in inset
plt.axes([0.4, 0.4, 0.5, 0.5])
Gcc = G.subgraph(sorted(nx.connected_components(G), key=len, reverse=True)[0])
pos = nx.spring_layout(G)
plt.axis("off")
nx.draw_networkx_nodes(G, pos, node_size=20)
nx.draw_networkx_edges(G, pos, alpha=0.4)
plt.show()

In [ ]:
plt.hist(G.degree())

In [ ]:
def plot_degree_dist(G):
    degrees = [G.degree(n) for n in G.nodes()]
    plt.hist(degrees)
    plt.title('Degree Distribution')
    plt.xlabel('Degrees')
    plt.ylabel('Number of Nodes')
    plt.show()
    
plot_degree_dist(G)

In [ ]:
df_test = pd.read_csv('test.csv',sep=',')
df_test = df_test.sample(n=5000)
df_test.drop(columns=['Unnamed: 0','id'],inplace=True)
df_test.Gender = df_test.Gender.apply(lambda x: 0 if x == 'Male' else 1)
df_test['Customer Type'] = df_test['Customer Type'].apply(lambda x: 0 if x == 'Loyal Customer' else 1)
df_test['Type of Travel'] = df_test['Type of Travel'].apply(lambda x: 0 if x == 'Personal Travel' else 1)
df_test['Class'] = df_test['Class'].apply(lambda x: 0 if x == 'Eco' else(1 if x == 'Eco Plus' else 2))
df_test['satisfaction'] = df_test['satisfaction'].apply(lambda x: 0 if x == 'neutral or dissatisfied' else 1)
df_test.reset_index(drop=True,inplace=True)
df_test.drop(columns=['Arrival Delay in Minutes'],inplace = True)


In [ ]:
X = df_test
X.Age = X.Age.apply(lambda x: 0 if x < 26 else(1 if x < 39 else(2 if x < 50 else 3)))
genvars = X[['Gender','Customer Type','Age','Type of Travel','Class']]

In [ ]:
x = genvars.to_numpy()
x = x.astype(int)
x = x.T

P_u_v = np.empty((len(x.T),len(x.T)))
prob_u_v = np.empty((len(x),1))
u = np.empty((len(x),1))
v = np.empty((len(x),1))

# create probability for connection between u and v
for i in range(len(x.T)):
    u = x[:,i]

    for k in range(len(x.T)):
        v = x[:,k]

        for j in range(len(x)):
            aff_current = aff[j]
            prob_u_v[j] = aff_current[u[j],v[j]]

        P_u_v[i,k] = np.prod(prob_u_v)

In [ ]:
A = np.triu(P_u_v,0)

for i in range(len(P_u_v)):
    for j in range(len(P_u_v)):
        if A[i,j] > np.random.rand():
            A[i,j] = 1
        else:
            A[i,j] = 0

In [ ]:
A = A + A.T - np.diag(np.diag(A))

In [ ]:
G_test = nx.from_numpy_matrix(A)

In [ ]:
features = pd.Series(df_test.satisfaction) # invested or inv_pref

In [ ]:
G_dgl_test = dgl.from_networkx(G_test)
feat = df_test.drop(columns=['satisfaction'])
feat = pd.get_dummies(feat)
scale = StandardScaler()
feat = pd.DataFrame(scale.fit_transform(feat))

embedding = nn.Parameter(torch.tensor(feat.values,dtype=torch.float))

# embed = nn.Embedding(102, 5)  # 34 nodes with embedding dim equal to 5
G_dgl_test.ndata['feat'] =  embedding
#G_dgl.ndata['train_mask'] = torch.zeros(len(G_dgl.nodes()), dtype=torch.bool).bernoulli(0.2)
G_dgl_test.ndata['label'] = torch.tensor(features)

In [ ]:
node_features = G_dgl_test.ndata['feat']
node_labels = G_dgl_test.ndata['label']
#train_mask = G_dgl.ndata['train_mask']
#valid_mask = G_dgl.ndata['val_mask']
n_features = node_features.shape[1]
n_labels = int(node_labels.max().item() + 1)

In [ ]:
def evaluate(model, graph, features, labels):
    model.eval()
    with torch.no_grad():
        logits = model(graph, features)
        logits = logits
        labels = labels
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [ ]:
model = SAGE(in_feats=n_features, hid_feats=50, out_feats=n_labels)
model.train()
logits = model(G_dgl_test, node_features)
loss = F.cross_entropy(logits, node_labels)
acc = evaluate(model, G_dgl_test, node_features, node_labels)


In [ ]:
acc

In [ ]:
val_tens = torch.tensor(features)
_, indices_val = torch.max(logits, dim=1)
val_lab = val_tens

print('Classification Accuracy validation set {}'.format(accuracy_score(val_lab,indices_val)))
print('-------------------------------------------------------------')
print('Confusion Matrix validation set:')
print(confusion_matrix(val_lab, indices_val))
